In [1]:
import gradio as gr
import tensorflow as tf

In [2]:
model= tf.keras.models.load_model('digit.keras')

In [3]:
import numpy as np
from PIL import Image, ImageOps

def recognize_digit(image):
    # Handle Gradio Sketchpad dict format
    if isinstance(image, dict):
        image = image.get("composite", None)
        if image is None:
            return {}

    if image is None:
        return {}

    # Convert numpy array to PIL, always resize to 28x28
    if isinstance(image, np.ndarray):
        pil_img = Image.fromarray(image.astype("uint8"), mode="L").resize((28, 28))
    elif isinstance(image, Image.Image):
        pil_img = image.convert("L").resize((28, 28))
    else:
        return {}

    arr = np.array(pil_img)
    if arr.mean() > 127:
        pil_img = ImageOps.invert(pil_img)

    img_array = np.array(pil_img).astype(np.float32) / 255.0
    img_array = img_array.reshape(1, 28, 28, 1)  # <- This shape works for your model!

    predictions = model.predict(img_array)
    if predictions.shape[0] == 1:
        predictions = predictions[0]
    return {str(i): float(predictions[i]) for i in range(10)}

In [4]:
iface = gr.Interface(
    fn=recognize_digit,
    inputs=gr.Sketchpad(canvas_size=(280, 280), image_mode="L"),
    outputs=gr.Label(num_top_classes=3),
    live=True
)
iface.launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 377ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
